In [ ]:
from IPython.display import display, Markdown, HTML
import pandas as pd
import numpy as np

import fastf1 as ff1

ff1.Cache.enable_cache("./data/cache")

YEAR = 2022
RACE = "Austria"

session = ff1.get_session(YEAR, RACE, "Race")
session.load(telemetry=True, laps=True, messages=True, weather=False)


In [ ]:

def get_test_stuff():
    with open("./notebooks/test.txt") as f:
        lines = f.readlines()

    woo = {}
    for l in lines:
        parts = l.strip().split(" ")
        lap_number = parts[1]
        driver_name = parts[2]
        passed_name = parts[4]
        position = parts[-1]

        driver = drivers[drivers["LastName"] == driver_name].iloc[0]
        passed = drivers[drivers["LastName"] == passed_name].iloc[0]
        driver_number = driver["DriverNumber"]

        if driver_number not in woo:
            woo[driver_number] = []
        woo[driver_number].append((lap_number, passed["Code"], position))

display(get_test_stuff())


In [79]:
from scrape.core import get_drivers
from scrape.core.driver import add_driver_code
from scrape.race import get_driver_overtakes
from scrape.race.timing import get_timing_data, add_lap_number_to_timing_data
from scrape.race.overtake import (
    __get_possible_overtakes,
    __get_position_changes,
)

DRIVER = "22"

drivers = get_drivers(YEAR)
timing = get_timing_data(session)
laps = session.laps.pick_driver(DRIVER)

overtakes = get_driver_overtakes(session, timing, DRIVER)
overtakes = add_driver_code(overtakes, drivers)
overtakes = add_driver_code(overtakes, drivers, suffix="Behind")
overtakes["LapNumber"] = overtakes["LapNumber"].astype('int64')
overtakes = overtakes.sort_values(by="LapNumber")
overtakes = overtakes[["LapNumber", "PassingStatus", "Code", "Position", "CodeBehind"]]
# display(overtakes)

overtakes_filtered = overtakes[overtakes["PassingStatus"].isnull()]
display(overtakes_filtered)

# possible_overtakes = __get_possible_overtakes(laps, timing)
# possible_overtakes = add_driver_code(possible_overtakes, drivers)
# possible_overtakes = add_driver_code(possible_overtakes, drivers, suffix="Behind")
# possible_overtakes["LapNumber"] = possible_overtakes["LapNumber"].astype('int64')
# possible_overtakes = possible_overtakes.sort_values(by="LapNumber")
# possible_overtakes = possible_overtakes[["Time", "LapNumber", "Position", "PositionBefore", "CodeBehind"]]
# display(possible_overtakes)

# possible_overtakes_b4 = __get_possible_overtakes_b4(laps, timing)
# possible_overtakes_b4 = add_driver_code(possible_overtakes_b4, drivers)
# possible_overtakes_b4 = add_driver_code(possible_overtakes_b4, drivers, suffix="Behind")
# # possible_overtakes_b4["LapNumber"] = possible_overtakes_b4["LapNumber"].astype('int64')
# possible_overtakes_b4 = possible_overtakes_b4.sort_values(by="LapNumber")
# possible_overtakes_b4 = possible_overtakes_b4[["Time", "LapNumber", "Position", "PositionBefore", "CodeBehind"]]
# display(possible_overtakes_b4)

# position_changes_b4 = __get_position_changes_b4(timing.query(f'DriverNumber == "{DRIVER}"').reset_index(drop=True))
# position_changes_b4 = add_lap_number_to_timing_data(laps.reset_index(drop=True), position_changes_b4)
# position_changes_b4 = add_driver_code(position_changes_b4, drivers)
# position_changes_b4["Time"] = position_changes_b4["Time"].astype('timedelta64[s]')
# position_changes_b4 = position_changes_b4[
#     ["Time", "LapNumber", "Code", "Position"]
# ]
# display(position_changes_b4)
# 
position_changes = __get_position_changes(laps, timing)
position_changes = add_driver_code(position_changes, drivers)
position_changes = add_driver_code(position_changes, drivers, suffix="Behind")
position_changes = add_driver_code(position_changes, drivers, suffix="Ahead")
position_changes = add_driver_code(position_changes, drivers, suffix="Passed")
# position_changes["Time"] = position_changes["Time"].astype('timedelta64[s]')
position_changes = position_changes[
    ["Time", "LapNumber", "Code", "CodePassed", "Position", "PositionBefore", "PositionGained", "CodeBehind", "CodeAhead"]
]
display(position_changes)



api            INFO 	Using cached data for timing_data


,LapNumber,PassingStatus,Code,Position,CodeBehind
1,2,NaN,VET,17,ALO
12,29,NaN,VET,15,LAT
14,37,NaN,VET,13,BOT
19,48,NaN,VET,16,ZHO
21,67,NaN,VET,16,TSU


,Time,LapNumber,Code,CodePassed,Position,PositionBefore,PositionGained,CodeBehind,CodeAhead
0,0 days 00:00:03.773000,NaN,VET,NaN,18,NaN,NaN,NaN,NaN
1,0 days 01:02:23.919000,NaN,VET,LAT,17,18,1.0,LAT,ALB
2,0 days 01:02:37.417000,NaN,VET,NaN,18,17,-1.0,ALO,LAT
3,0 days 01:02:46.456000,NaN,VET,LAT,17,18,1.0,LAT,TSU
4,0 days 01:03:01.725000,NaN,VET,PER,16,17,1.0,PER,TSU
5,0 days 01:03:12.016000,NaN,VET,NaN,17,16,-1.0,ALO,LAT
6,0 days 01:03:53.573000,2,VET,NaN,18,17,-1.0,BOT,ALO
7,0 days 01:04:01.850000,2,VET,ALO,17,18,1.0,ALO,LAT
8,0 days 01:04:06.698000,2,VET,NaN,18,17,-1.0,BOT,ALO
9,0 days 01:04:16.591000,2,VET,ALO,17,18,1.0,ALO,LAT
